# PART I: Creating the Study's Datasets

# 0. Setup

Before we begin, make sure you have installed all the required Python packages. (The instructions below use pip. You can use easy_install, too.) Also, consider using virtualenv for a cleaner installation experience instead of sudo. I also recommend to running the code via IPython Notebook.
* pip install --upgrade turicreate
* pip install --upgrade networkx
* pip install --upgrade pymongo



Please download the KDD Cup 2016 data, and please also download the project files from our GitHub repository. Through this research, we use the various constants that appear in consts.py. Please change the DATASETS_AMINER_DIR, DATASETS_BASE_DIR, and SFRAMES_BASE_DIR to your local directories, where you can download the datasets and save the project's SFrames.

**Note: Creating the following SFrame requires considerable computation power for long periods.** 

In [1]:
%load_ext autoreload
%autoreload 2
%aimport
%matplotlib inline

Modules to reload:
all-except-skipped

Modules to skip:



In [5]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [6]:
from tqdm import tqdm_notebook as tqdm 

# 1. Creating the SFrames

In this study, we used the following datasets:
* [The Microsoft Academic KDD Cup 2016 dataset](https://kddcup2016.azurewebsites.net/Data) - The Microsoft Academic KDD Cup Graph dataset (referred to as the MAG 2016 dataset) contains data on over 126 million papers. The main advantage of this dataset is that it has undergone several preprocessing iterations of author entity matching (any author is identified by ID) and paper deduplication. Additionally, the dataset match between papers and their fields of study includes the hierarchical structure and connections between various fields of study.  <br/>  The link is dead...

* [AMiner dataset](https://aminer.org/open-academic-graph) - The AMiner dataset contains information on over 154 million papers collected by the AMiner team. The dataset contains papers' abstracts, ISSNs, ISBNs, and details on each paper. <br/>  Cureently there is a V2 and to download which one...

* [SJR dataset](http://www.scimagojr.com/journalrank.php) -  The SCImago Journal Rank open dataset (referred to as the SJR dataset) contains journals and country specific metric data starting from 1999. In this study, we used the SJR dataset to better understand how various journal metrics have changed over time. <br/>  How to download?


## 1.1 The Microsoft Academic Dataset

The first step is to convert the dataset text files into SFrame objects using the code located under the SFrames creator directory, using the following code.

In [63]:
from ScienceDynamics.datasets.microsoft_academic_graph import MicrosoftAcademicGraph
from ScienceDynamics.config.configs import DATASETS_BASE_DIR
mag = MicrosoftAcademicGraph(DATASETS_BASE_DIR)

In [14]:
import pandas as pd

In [15]:
paper_author_affiliations = pd.read_csv("/storage/homedir/dima/.scidyn2/MAG/PaperAuthorAffiliations.txt.gz", sep="\t", names=["PaperId", "AuthorId", "AffiliationId", "AuthorSequenceNumber", "OriginalAuthor", "OriginalAffiliation"])

In [18]:
sf = SFrame(paper_author_affiliations.replace({pd.np.nan: None}))

In [19]:
sf.save("/storage/homedir/dima/.scidyn2/MAG/sframesPaperAuthorAffiliations.sframe")

In [21]:
!mv  /storage/homedir/dima/.scidyn2/MAG/sframesPaperAuthorAffiliations.sframe /storage/homedir/dima/.scidyn2/MAG/sframes/PaperAuthorAffiliations.sframe

In [ ]:
["PaperId", "Rank", "Doi", "DocType", "PaperTitle", "OriginalTitle", "BookTitle", "Year", "Date",
                "Publisher", "JournalId", "ConferenceSeriesId", "ConferenceInstanceId", "Volume", "Issue", "FirstPage",
                "LastPage", "ReferenceCount", "CitationCount", "EstimatedCitation", "OriginalVenue", "FamilyId",
                "CreatedDate"]

In [7]:
mag.fields_of_study_papers_ids()

Unable to parse line "140557674	25673			tassa kohdassa heterorouva nyokkaa nakokulma naisparien henkilohaastatteluissa	"Tässä kohdassa heterorouva nyökkää.℗» : Näkökulma naisparien henkilöhaastatteluissa		2007	2007-01-01	Helsinki : Kotikielen seura				111	1	68		0	0	0..."

Unable to parse line "140830831	21272			parametric trusses an android app to understand structural design	"PARAMETRIC TRUSSES”, AN ANDROID APP TO UNDERSTAND STRUCTURAL DESIGN		2014	2014-01-01	IATED						4347	4356	0	1	1	INTED2014 Proceedings	2016-06-24"

2 lines failed to parse correctly

Finished parsing file /storage/homedir/dima/.scidyn2/MAG/Papers.txt.gz

Parsing completed. Parsed 100 lines in 2.32794 secs.

...

Read 213577014 lines. Lines per second: 94552.5

6824 lines failed to parse correctly

Finished parsing file /storage/homedir/dima/.scidyn2/MAG/Papers.txt.gz

Parsing completed. Parsed 214006156 lines in 2263.97 secs.

RuntimeError: Unable to interpret value of "Journal" as a integer.

The above two lines of code will create a set of SFrames with all the dataset data. The SFrames will include data on authors’ papers, keywords, fields of study, and more. Moreover, the code will construct the Extended Papers SFrame, which contains various meta data on each paper in the dataset.

In [ ]:
sframe_list = [ self.papers_citation_number_by_year, 
                        self.urls]

In [14]:
p = mag.papers

In [15]:
os.path.exists("/storage/homedir/dima/.scidyn/MAG/sframes/ExtendedPapers.sframe")

True

In [31]:
mag_sf = mag.extended_papers


In [34]:
col= 'Fields of study parent list (L1)'

In [36]:
new_col_name = "Field ID"
sf = mag_sf.stack(col, new_column_name=new_col_name)

In [55]:
mag.fields_of_study_papers_ids_sframes()

Field of study ID,Paper IDs,Field of study name,Number of Paper,Level
01C396A7,"[7FE89521, 0E23C189,59E35B91, 587AF2A4, ...",Media studies,24854,1
02005B3E,"[7887D4CE, 7DAF4838,7919A107, 5E1E0717, ...",Calculus,181633,1
0A778812,"[7EAF768F, 7DC84FFC,49816720, 7DF2E00F, ...",Natural resourceeconomics ...,1222,1
0BE4BA29,"[7CFE299E, 5E628D73,80BEF0CE, 7E555D49, ...",Law,1343748,1
014EF258,"[7F9691D6, 7DED9FA0,7E4483E2, 7DFA3CA8, ...",Combinatorial chemistry,42382,1
023E10AF,"[7FD252F0, 84287EA0,728CDCF0, 5C37DA5F, ...",Agricultural science,49446,1
02C0117D,"[790C14A8, 5DD62343,80D484FA, 78CD50CE, ...",Nuclear magneticresonance ...,486056,1
06E88D7C,"[7CFE299E, 79F82B7D,7E087EF0, 5C65E96E, ...",Software Engineering,323620,1
073136E6,"[0A8B0683, 77DE62B4,5ABC5929, 5E494EDA, ...",Optics,888483,1
08ED7E6D,"[7CC75151, 75AE9967,58AF9956, 5B9DDC2F, ...",Econometrics,41660,1


In our study, we also analyzed how various authors' attributes, such as the number of published papers, number of coauthors, etc., has changed over time. To achieve this, we created an authors features SFrame using the following code:

In [8]:
mag.author_names

Finished parsing file /storage/homedir/dima/.scidyn/MAG/Authors.txt

Parsing completed. Parsed 100 lines in 5.93314 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 2100324 lines. Lines per second: 346187

Read 12600385 lines. Lines per second: 1.08461e+06

...

Using default 64 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 128)"

Note that increasing the degree of parallelism also increases the memory footprint.

Author ID,Author name,First name,Last name
00001F05,nancy praill,nancy,praill
000025C5,david banitt,david,banitt
00002AD3,david s rebergen,david,rebergen
000038C0,西原 一幸,西原,一幸
000060E7,francesco saveriointorcia ...,francesco,intorcia
00006A31,b zelazowska,b,zelazowska
00009F6B,bo glimskar,bo,glimskar
0000B5FA,lars goerigk,lars,goerigk
0000D3AB,default admin user,default,user
0000DF94,leila medjkoune,leila,medjkoune


In [6]:
import turicreate
turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 2)
turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_GRAPH_LAMBDA_WORKERS', 2)
from ScienceDynamics.datasets.mag_authors import AuthorsFeaturesExtractor
a = AuthorsFeaturesExtractor(mag)
# a.authors_features
#This need to run on a strong server and can take considerable time to run
# a_sf = a.get_authors_all_features_sframe()
# a_sf #the SFrame can be later loaded using tc.load_sframe(AUTHROS_FEATURES_SFRAME)

In [10]:
a.paper_authors_years

AuthorId,PaperId,Year
2151921381,37,2004
2722638736,37,2004
2767506227,37,2004
2126642415,108,2013
1995014452,125,1988
2002579779,125,1988
2250382311,125,1988
2283694448,125,1988
2110272823,147,2008
2165699729,147,2008


In [24]:
a.paper_author_affiliation_sframe

PaperId,AuthorId,AffiliationId,AuthorSequenceNumber,OriginalAuthor,OriginalAffiliation,Year
37,2151921381,30338065.0,2,Helena Elisa Stein,Universidade de Caxias doSul ...,2004
37,2722638736,None,3,Lauro Machado Neto,None,2004
37,2767506227,30338065.0,1,Ana Paula TedescoGabrieli ...,Ortopedia e Traumatologia|Universidade de Caxias ...,2004
108,2126642415,165102784.0,1,Matthew Lovett,Duquesne University,2013
125,1995014452,169199633.0,2,David Hannaford,Aston University and BISApplied Systems Ltd ...,1988
125,2002579779,169199633.0,1,Jon Bader,Aston University and BISApplied Systems Ltd ...,1988
125,2250382311,169199633.0,4,John Edwards,Aston University and BISApplied Systems Ltd ...,1988
125,2283694448,169199633.0,3,Alastair Cochran,Aston University and BISApplied Systems Ltd ...,1988
147,2110272823,None,2,J.M. García Puga,None,2008
147,2165699729,None,1,J. E. Callejas Pozo,None,2008


In [11]:
tc.config.get_runtime_config()


{'TURI_CACHE_FILE_HDFS_LOCATION': '',
 'TURI_CACHE_FILE_LOCATIONS': '/storage/homedir/dima/.scidyn2/tmp',
 'TURI_DEFAULT_NUM_GRAPH_LAMBDA_WORKERS': 12,
 'TURI_DEFAULT_NUM_PYLAMBDA_WORKERS': 2,
 'TURI_FAST_COMPACT_BLOCKS_IN_SMALL_SEGMENT': 8,
 'TURI_FILEIO_ALTERNATIVE_SSL_CERT_DIR': '',
 'TURI_FILEIO_ALTERNATIVE_SSL_CERT_FILE': '/storage/homedir/dima/miniconda3/envs/promed/lib/python3.6/site-packages/certifi/cacert.pem',
 'TURI_FILEIO_INSECURE_SSL_CERTIFICATE_CHECKS': 0,
 'TURI_FILEIO_MAXIMUM_CACHE_CAPACITY': 270489230336,
 'TURI_FILEIO_MAXIMUM_CACHE_CAPACITY_PER_FILE': 270489230336,
 'TURI_FILEIO_MAX_REMOTE_FS_CACHE_ENTRIES': 0,
 'TURI_FORCE_IPC_TO_TCP_FALLBACK': 0,
 'TURI_GLOBALS_PYTHON_EXECUTABLE': '/storage/homedir/dima/miniconda3/envs/promed/bin/python',
 'TURI_LAMBDA_WORKER_CONNECTION_TIMEOUT': 60.0,
 'TURI_MIN_SECONDS_BETWEEN_TICK_PRINTS': 3.0,
 'TURI_ML_DATA_STATS_PARALLEL_ACCESS_THRESHOLD': 1048576,
 'TURI_ML_DATA_TARGET_ROW_BYTE_MINIMUM': 262144,
 'TURI_NUM_GPUS': -1,
 'TURI_S

In [ ]:
a.get_co_authors_dict_sframe()

2020-04-02 15:34:12,640 [MainThread  ] [INFO ]  Calcualting authors' coauthors by year


In [ ]:
a.get_authors_papers_dict_sframe()

In [9]:
self = a

In [10]:
p_sf = self._p_sf[['PaperId']]  # 22082741
a_sf = self._mag.paper_author_affiliations["AuthorId", "PaperId"]
a_sf = a_sf.join(p_sf, on="PaperId")
a_sf = a_sf[["AuthorId"]].unique()
g = self.get_authors_papers_dict_sframe()
a_sf = a_sf.join(g, on="AuthorId", how="left")  # 22443094 rows
a_sf.__materialize__()
del g
del p_sf
g = self.get_co_authors_dict_sframe()

2020-03-02 10:51:11,648 [MainThread  ] [INFO ]  Calcualting authors' papers by year
2020-03-02 11:31:19,056 [MainThread  ] [INFO ]  Calcualting authors' coauthors by year


RuntimeError: vector::_M_default_append

In [13]:
from turicreate import aggregate as agg

In [15]:
# print("Calcualting authors' coauthors by year")
# sf = self.paper_authors_years
# sf = sf.join(sf, on='PaperId')
# sf2 = sf[sf['AuthorId'] != sf['AuthorId.1']]
# sf2 = sf2.remove_column('Year.1')
# sf2.__materialize__()
# g = sf2.groupby(['AuthorId', 'Year'], {'Coauthors List': agg.CONCAT('AuthorId.1')})
# del sf
# g.__materialize__()
# del sf2
g['Coauthors Year'] = g.apply(lambda r: (r['Year'], r['Coauthors List']))
g2 = g.groupby("AuthorId", {'Coauthors list': agg.CONCAT('Coauthors Year')})
g2['Coauthors by Years Dict'] = g2['Coauthors list'].apply(lambda l: {y: coa_list for y, coa_list in l})
g2 = g2.remove_column('Coauthors list')


MemoryError: std::bad_alloc

In [7]:
import turicreate as tc
tc.__version__

'6.1'

The above SFrame contains various features of each author that were constructed based on analyzing the author’s papers that have at least 5 references. If you notice, the author’s SFrame contains each author’s gender prediction. This column was created by obtaining first-name gender statistics from the [SSA Baby Names](http://www.ssa.gov/oact/babynames/names.zip]) and [WikiTree](https://www.wikitree.com/wiki/Help:Database_Dumps) datasets which include over 115 thousands unique first names (see details in geneder_classifier.py). 

## 1.2 The AMiner Dataset

After downloading the [AMiner website](https://aminer.org/open-academic-graph), simply load to an SFrame using the following code:

In [17]:
from ScienceDynamics.datasets.aminer import Aminer
from ScienceDynamics.config.configs import DATASETS_AMINER_DIR

In [8]:
1

1

In [18]:
a = Aminer(DATASETS_AMINER_DIR)

In [9]:
a.data

Finished parsing file /storage/homedir/dima/.scidyn/AMiner/aminer_papers_30.txt

Parsing completed. Parsed 100 lines in 1.07869 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 33536 lines. Lines per second: 22892.2

Read 476080 lines. Lines per second: 68313.8

...

Read 154247383 lines. Lines per second: 90263.7

Read 154591163 lines. Lines per second: 90198.8

Finished parsing file /storage/homedir/dima/.scidyn/AMiner/aminer_papers_8.txt

Parsing completed. Parsed 154771161 lines in 1716.82 secs.

abstract,authors,doi,id
None,"[{'name': 'Ö. Tan'},{'name': 'A. Sahin ...",10.3989/mc.2007.v57.i288.67 ...,53e9b2dbb7602d9703d8c652
None,"[{'name': 'A. Goldman'},{'name': 'D. G. ...",10.1364/AO.19.003721,53e9b2dbb7602d9703d8c653
Objective:It summarizednursing measures for ...,"[{'name': 'Zhang Rong','org': 'Affiliated ...",None,53e9b2dbb7602d9703d8c654
None,"[{'name': 'Elisabeth vander Linden'}, {'name': ...",10.1075/eurosla.5.07lin,53e9b2dbb7602d9703d8c655
None,"[{'name': 'N. Cordes'},{'name': 'L. Plasswil ...",10.1016/S0959-8049(01)81239-9 ...,53e9b2dbb7602d9703d8c656
None,"[{'name': 'ShigeruIkeda'}, {'name': ...",10.1021/cm970221c,53e9b2dbb7602d9703d8c657
Bovine chondrocytes wereisolated from the ...,"[{'name': 'Takashi Sato','org': 'Biomaterials ...",10.1016/j.msec.2003.12.010 ...,53e9b2dbb7602d9703d8c658
"Word count: 4667 words(including abstract, key ...","[{'name': 'EnnioCascetta'}, {'name': ...",None,53e9b2dbb7602d9703d8c659
Abstract—A novel familyof low-density parity- ...,"[{'name': 'I. B.Djordjevic'}, {'name': ...",10.1109/LCOMM.2004.833833,53e9b2dbb7602d9703d8c65b
None,"[{'name': 'DANIEL M. JOEL'}, {'name': 'V ITALY ...",None,53e9b2dbb7602d9703d8c65c


## 1.3 The SJR Dataset

First, we download all the journal ranking files from [the SJR website](http://www.scimagojr.com/journalrank.php).
Next, we use the following code to create a single SFrame with all the journal data:

In [19]:
from ScienceDynamics.datasets.sjr import SJR
from ScienceDynamics.config.configs import DATASETS_SJR_DIR
sjr = SJR(DATASETS_SJR_DIR)

In [4]:
sjr.data

Finished parsing file /storage/homedir/dima/.scidyn/sjr/scimagojr 1999.csv

Parsing completed. Parsed 100 lines in 0.152944 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,str,str,str,str,int,int,int,int,int,int,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /storage/homedir/dima/.scidyn/sjr/scimagojr 1999.csv

Parsing completed. Parsed 17166 lines in 0.109449 secs.

...

Using default 12 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 24)"

Note that increasing the degree of parallelism also increases the memory footprint.

Rank,Sourceid,Title,Type,SJR,SJR Best Quartile,H index,Total Docs.,Total Docs. (3years)
1,16801,Annual Review ofBiochemistry ...,journal,"50,518",Q1,268,30,81
1,16801,Annual Review ofBiochemistry ...,journal,"50,518",Q1,268,30,81
2,18434,Cell,journal,"43,449",Q1,682,354,1359
2,18434,Cell,journal,"43,449",Q1,682,354,1359
3,20651,Annual Review ofImmunology ...,journal,"43,020",Q1,274,29,81
3,20651,Annual Review ofImmunology ...,journal,"43,020",Q1,274,29,81
4,18395,Annual Review of Cell andDevelopmental Biology ...,book series,"35,051",Q1,199,25,61
4,18395,Annual Review of Cell andDevelopmental Biology ...,book series,"35,051",Q1,199,25,61
5,14181,Annual Review ofNeuroscience ...,book series,"25,760",Q1,217,21,60
6,22126,Genes and Development,journal,"25,272",Q1,401,306,915


## 1.4 Joint Datasets

The MAG and AMiner datasets have a slightly different set of features. While the MAG dataset contains data on each author with a unique author ID, the AMiner contains additional data on each paper, including the paper's abstract and the paper's ISSN or ISBN. Additionally, the SJR dataset contains data about each journal's ranking.

To combine the data from the author publication record and the journals' rankings, we join the datasets. First, we joined the MAG and AMiner datasets by matching DOI values, using the following code (see also create_mag_aminer_sframe.py):

In [6]:
from ScienceDynamics.datasets.joined_dataset import JoinedDataset
from ScienceDynamics.config.configs import DATASETS_BASE_DIR, DATASETS_SJR_DIR, DATASETS_AMINER_DIR, STORAGE_PATH

In [7]:
jd = JoinedDataset(STORAGE_PATH, DATASETS_SJR_DIR, DATASETS_AMINER_DIR, mag_path=DATASETS_BASE_DIR/ "MicrosoftAcademicGraph.zip")

In [21]:
jd.mag._dataset_dir

PosixPath('/storage/homedir/dima/.scidyn/MAG')

In [175]:
from tqdm import tqdm_notebook as tqdm

In [20]:
print(b_lim, u_lim)


5865272 5865273


In [1]:
data[data["MAG Paper ID"]=='74024986']

NameError: name 'data' is not defined

In [22]:
sf[5865272]

{'Aminer Paper ID': '53e99d7ab7602d97026364e9',
 'Authors List Sorted': '["7D4BF247","766E9461","12B235CA"]',
 'Authors Number': 3.0,
 'Conference ID mapped to venue name': 'nan',
 'Field of study list': '["078EB9BF","07799FC6","1F9A2DF7","014CFD3E"]',
 'Field of study list names': '["Topology","Encoding",,"Antenna"]',
 'Fields of study parent list (L0)': '["073B64E4","0205A1DB","00F03FC7"]',
 'Fields of study parent list (L1)': '["078EB9BF","00137C13"]',
 'Fields of study parent list (L2)': '["08F300F3"]',
 'Fields of study parent list (L3)': '["07799FC6","014CFD3E"]',
 'Fields of study parent list names (L0)': '["Physics","Mathematics","Psychology"]',
 'Fields of study parent list names (L1)': '["Topology","Astronomy"]',
 'Fields of study parent list names (L2)': '["Radio astronomy"]',
 'Fields of study parent list names (L3)': '["Encoding","Antenna"]',
 'Journal ID mapped to venue name': '0044B422',
 'Keywords List': '["topology","encoding","integrated circuits","antennas"]',
 'MAG 

In [181]:
data.save("data.csv","csv")

In [209]:
from turicreate import SFrame

In [7]:
import pandas as pd

In [211]:
data.to_csv("data2.csv", encoding='utf-8')

In [8]:
data = pd.read_csv("data.csv",encoding='latin1',error_bad_lines=False)

/storage/homedir/dima/miniconda3/envs/promed/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,7,8,9,31,32,33,37,38,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
len(sf),len(jd.aminer_mag_links_by_doi)

(29070469, 29070469)

In [258]:
data.head(10)["MAG Paper ID"].str.encode('utf-8')

0    b'01B27BE8'
1    b'027D0030'
2    b'7CFE299E'
3    b'59BEBE1C'
4    b'5873C011'
5    b'0B00AFD8'
6    b'5C66D743'
7    b'040121AE'
8    b'584D8787'
9    b'58B384F6'
Name: MAG Paper ID, dtype: object

In [11]:
for col in tqdm(data.columns):
    if data[col].dtype == object:
        print(col)
        data[col] = data[col].str.encode('utf-8')

MAG Paper ID
Original paper title
Normalized paper title
Paper publish date
Paper Document Object Identifier (DOI)
Original venue name
Normalized venue name
Journal ID mapped to venue name
Conference ID mapped to venue name
Total Citations by Year
Total Citations by Year without Self Citations
Authors List Sorted
Keywords List
Field of study list
Field of study list names
Fields of study parent list (L0)
Fields of study parent list names (L0)
Fields of study parent list (L1)
Fields of study parent list names (L1)
Fields of study parent list (L2)
Fields of study parent list names (L2)
Fields of study parent list (L3)
Fields of study parent list names (L3)
Urls
abstract
authors
Aminer Paper ID
isbn
issn
issue
keywords
lang
page_end
page_start
pdf
references
title
url
venue
volume



In [9]:
from turicreate import SFrame, load_sframe

In [18]:
sf = SFrame(data_dict)

In [21]:
sf.save("temp.sframe")

In [10]:
sf = load_sframe("temp.sframe")

In [11]:
sf = sf.unpack('X1',column_name_prefix="")

In [19]:
len(sf)

29070469

In [12]:
data_dict = data.to_dict(orient='records')

In [20]:
del data_dict

In [14]:
len(data_dict)

29070469

In [168]:
b_lim, u_lim

(5865272, 5865273)

In [170]:
# data[5866103]
data[5865273]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 1247: invalid continuation byte

In [121]:
df = data.to_dataframe()

KeyboardInterrupt: 

In [164]:
x = jd.aminer_mag_links_by_doi[0:5866103].append(jd.aminer_mag_links_by_doi[5866128:])

'Ã±'

In [240]:
te = SFrame()
te["e"] = ["í".encode('latin-1').decode('latin-1').encode('utf-8')]

In [239]:
str("í".encode('latin-1'), 'utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 0: unexpected end of data

b'\xed'

In [68]:
jd._sframe_dir

PosixPath('/storage/homedir/dima/.scidyn/sframes')

In [14]:
sf.save(str(jd._sframe_dir/"PapersAMinerMagJoin.sframe"))

Using the joined dataset, we obtained an SFrame with the joint meta data of 28.9 million papers. We can take this SFrame and join it with the SJR dataset.

In [24]:
jd.aminer_mag_sjr(2015)

MAG Paper ID,Original paper title,Normalized paper title,Paper publish year,Paper publish date
80F8FCAF,Trust-assisted anomalydetection and ...,trust assisted anomalydetection and ...,2011,2011/06
7E7B8146,Incentive-aware datadissemination in delay- ...,incentive aware datadissemination in delay ...,2011,2011/06
77B641FC,MOVi+: Improving thescalability of mobile ...,movi improving thescalability of mobile ...,2012,2012/06
76E06216,Identifying infectionsources in large tree ...,identifying infectionsources in large tree ...,2012,2012/06
7DAFE959,Coping with packet replayattacks in wireless ...,coping with packet replayattacks in wireless ...,2011,2011/06
7E428CE1,An optimal distributedmalware defense system ...,an optimal distributedmalware defense system ...,2011,2011/06
762EDB3C,A privacy-preservingsocial-aware incentive ...,a privacy preservingsocial aware incentive ...,2012,2012/06
7E018A37,Adaptive energy-efficientspectrum probing in ...,adaptive energy efficientspectrum probing in ...,2011,2011/06
7DDBB811,Broadcasting in multichannel wireless netw ...,broadcasting in multichannel wireless netw ...,2011,2011/06
815AC7AF,A software-hardwareemulator for sensor ...,a software hardwareemulator for sensor ...,2011,2011/06


# 2. Loading the Dataset to MongoDB

Using Turicreate and SFrame objects can help us get general data on how academic publication dynamics have changed over time, but it would be challenging to use this data to create more complicated insights, such as the trends of a specific journal. To reveal more complicated insights using the data, we would need to load the dataset to a different framework. In this study, we chose to use MongoDB as our framework for more complicated queries.
We installed MongoDB on Ubuntu 17.10 using the instructions in the following [link](https://medium.com/gatemill/how-to-install-mongodb-3-6-on-ubuntu-17-10-ac0bc225e648). After MongoDB is installed and running, please remember to set the user and password, and update MONGO_HOST & MONGO_PORT vars in consts.py (one can also adjust the connection to include user password auth).
Now, the next step is to load the above created SFrames to collections in MongoDB using mongo_connecter.py:

In [ ]:
from ScienceDynamics.mongo_connector import load_sframes
load_sframes(mag,sjr,jd) #this will load the SFrame to a local

2019-05-01 08:43:05,560 [MainThread  ] [INFO ]  Loading authors features
2019-05-01 08:43:05,608 [MainThread  ] [INFO ]  Converting


Less than 64 successfully started. Using only 16 workers.

All operations will proceed as normal, but lambda operations will not be able to use all available cores.

Using default 64 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 128)"

Note that increasing the degree of parallelism also increases the memory footprint.

2019-05-01 08:52:09,766 [MainThread  ] [INFO ]  Inserting rows 0 - 100000 to journals.authors_features
2019-05-01 08:52:39,118 [MainThread  ] [INFO ]  Inserting rows 100000 - 200000 to journals.authors_features
2019-05-01 08:53:11,346 [MainThread  ] [INFO ]  Inserting rows 200000 - 300000 to journals.authors_features
2019-05-01 08:53:41,134 [MainThread  ] [INFO ]  Inserting rows 300000 - 400000 to journals.authors_features
2019-05-01 08:53:56,001 [MainThread  ] [INFO ]  Inserting rows 400000 - 500000 to journals.authors_features
2019-05-01 08:54:21,961 [MainThread  ] [INFO ]  Inserting rows 500000 - 600000 to journals.authors_features
2019-05-01 08:54:59,461 [MainThread  ] [INFO ]  Inserting rows 600000 - 700000 to journals.authors_features
2019-05-01 08:55:28,084 [MainThread  ] [INFO ]  Inserting rows 700000 - 800000 to journals.authors_features
2019-05-01 08:55:47,462 [MainThread  ] [INFO ]  Inserting rows 800000 - 900000 to journals.authors_features
2019-05-01 08:56:07,444 [MainThre

In the end of the loading process, six collections will be loaded to the journal database.

In [4]:
from ScienceDynamics.mongo_connector import MongoDBConnector
MD = MongoDBConnector()

In [6]:
MD.client.journals.collection_names()

/storage/homedir/dima/miniconda3/envs/promed/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


[]

In [61]:
for i, item in enumerate(MD.client.journals.aminer_mag_papers.find()):
    print(item)
    if i >4:
        break

{'_id': ObjectId('5cca685465387acb12698540'), 'Aminer Paper ID': '555045bc45ce0a409eb58f90', 'Authors List Sorted': '["834A11E2","7E8BA14F","852B2668"]', 'Authors Number': 3.0, 'Conference ID mapped to venue name': '42D7146F', 'Field of study list': 'nan', 'Field of study list names': 'nan', 'Fields of study parent list (L0)': 'nan', 'Fields of study parent list (L1)': 'nan', 'Fields of study parent list (L2)': 'nan', 'Fields of study parent list (L3)': 'nan', 'Fields of study parent list names (L0)': 'nan', 'Fields of study parent list names (L1)': 'nan', 'Fields of study parent list names (L2)': 'nan', 'Fields of study parent list names (L3)': 'nan', 'Journal ID mapped to venue name': 'nan', 'Keywords List': 'nan', 'MAG Paper ID': '01B27BE8', 'Normalized paper title': 'evaluating polarity for verbal phraseological units', 'Normalized venue name': 'micai', 'Original paper title': 'Evaluating Polarity for Verbal Phraseological Units', 'Original venue name': 'mexican international confe

In [216]:
sf = SFrame.read_csv("data2.csv")

Finished parsing file /storage/homedir/dima/Projects/ScienceDynamics/examples/data2.csv

Parsing completed. Parsed 100 lines in 0.197279 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,str,int,str,str,str,str,str,str,int,int,dict,dict,list,float,list,list,list,list,list,list,list,list,list,list,list,list,str,list,str,str,str,str,list,str,float,int,int,str,list,str,list,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Unable to parse line "2816,788F5CEE,The design and the development of the PKM based on the Android platform,the design and the development of the pkm based on the android platform,2010,2010/10,10.1109/ICINA.2010.5636964,international conference on information networking,icoin,,..."

Unable to parse line "2708,787207D6,"Comparison of PCA, LDA and GDA for palmprint verification",comparison of pca lda and gda for palmprint verification,2010,2010/10,10.1109/ICINA.2010.5636417,international conference on information networking,icoin,,43F6AF91,19555,0,,,"[""7F92..."

...

Read 28472146 lines. Lines per second: 20017.6

Read 28577752 lines. Lines per second: 20014.8

431446 lines failed to parse correctly

Finished parsing file /storage/homedir/dima/Projects/ScienceDynamics/examples/data2.csv

Parsing completed. Parsed 28639023 lines in 1431.51 secs.

In [27]:
MD.client.journals.aminer_mag_papers.count()

/storage/homedir/dima/miniconda3/envs/promed/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


29070469

In [7]:
MD.client.journals.aminer_mag_papers.remove()

/storage/homedir/dima/miniconda3/envs/promed/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{'ok': 1, 'n': 0}

In the second part of the tutorial, we will demonstrate how the above created MongoDB collections can be utilized to calculate various statistics on paper collections, authors, journals, and research domains.